In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [2]:
train1  = pd.read_csv('../Data/train_2.csv')

Generate Page features

In [10]:
a = train1.Page.str.split('mediawiki.org|wikimedia.org|wikipedia.org',expand=True)
b = np.where(a.loc[:,2].isnull(),a.loc[:,1],a.loc[:,2])
_,index_array = np.unique(b,return_inverse=True)
website_array = [train1.Page.str.contains(web).values.astype(np.int8) \
                 for web in ['mediawiki.org','wikimedia.org','wikipedia.org']] # website
lang_array = train1.Page.str.extract('([a-z]{2}).wikipedia').fillna('nan').values.flatten()
_,lang_array = np.unique(lang_array,return_inverse=True) # language

In [25]:
Page_X = \
np.concatenate([OneHotEncoder(dtype=np.float32,sparse=False,categories='auto').fit_transform(lang_array[:,np.newaxis]),\
                OneHotEncoder(dtype=np.float32,sparse=False,categories='auto').fit_transform(index_array[:,np.newaxis]),\
                np.stack(website_array,1)],1)

In [33]:
np.save('../Data/Page_X.npy',Page_X)

Save the rest

In [43]:
train1 = train1.iloc[:,1:]
IsMissing = train1.isnull().values
value = train1.T.fillna(pd.Series(np.nanmean(train1.values,1),train1.index),axis=0)
value = value.T.values.astype(np.float32)
start_index = np.argmin(IsMissing,1)
date_index=pd.to_datetime(train1.columns).to_series().reset_index(drop=True)

In [111]:
np.save('../Data/value.npy',value)
np.save('../Data/IsMissing.npy',IsMissing)
np.save('../Data/start_index.npy',start_index)

In [112]:
date_index.to_pickle('../Data/date_index')